```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2024
Profesor: Facundo Santiago
```

# Solución: ¿Qué tan memorable es un video?

Este notebook les permitirá tener a disposición todos los recursos que se vieron en durante la materia. Todos los fragmentos de código se descargaran utilizando la sección preparación del ambiente.

## Preparación del ambiente

### Sets de datos

In [1]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/ground_truth.csv --directory-prefix ./Data/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/audio_vectors.csv --directory-prefix ./Data/Features/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/c3d_vectors.csv --directory-prefix ./Data/Features/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/caption_features.csv --directory-prefix ./Data/Features/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/colors_features.csv --directory-prefix ./Data/Features/ --quiet

### Librerías de codigo que se utilizan durante el curso

In [4]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/solucion_base.txt --quiet
!pip install -r solucion_base.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.3 MB/s eta 0:00:00


Descargamos todo el código con utilidades que se vieron en el curso desde el repositorio de la materia:

In [9]:
!git clone -n --depth=1  https://github.com/santiagxf/M72109
!cd M72109 && git sparse-checkout set --no-cone m72109 && git checkout
!mv M72109/m72109 . && rm -rf M72109

Descargamos la etiquetas de YamNET en el caso de utilizarlas:

In [11]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/Models/yamnet/yamnet_class_map.csv \
    --directory-prefix ./Models/yamnet/ --quiet --no-clobber

Descargamos modelos de Word2Vec en ingles:

In [12]:
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model-en.bin --directory-prefix ./Models/Word2Vec --quiet

### Imports

In [19]:
import pandas as pd
import numpy as np
import librosa
import os
import soundfile as sf
import torch
import datasets
import transformers
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, SpatialDropout1D
import tensorflow_datasets as tfds

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from m72109.audio.plotting import plot_audio_embeddings
from m72109.nlp.normalization import TextNormalizer, TweetTextNormalizer
from m72109.nlp.transformation import Word2VecVectorizer, PadSequenceTransformer, split_text_with_context

import azure.cognitiveservices.speech as speechsdk
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

## Solución

Cuentan con 2 tipos de anotaciones para cada uno de los fragmentos de video disponibles:
 - **memorability_score:** Representa el puntaje de memorabilidad de la secuencia en particular, desde 0 a 1. Valores más grandes son mejores.
 - **memorable:** Variable categórica que representa si un video es memorable o no. Un video con `memorability_score` superior a `0.5` es marcado como memorable (`1`), sino es marcado como no memorable (`0`)

In [15]:
labels = pd.read_csv('Data/ground_truth.csv')

In [16]:
labels.head(2)

,movie_name,start(sec),end(sec),sequence_name,Neutral (1)_Typical (0),nb_annotations,memorability_score,memorable
0,127 hours,2000,2010,127_hours_2000_2010_1,0,5,1.0,1
1,127 hours,2182,2192,127_hours_2182_2192_5,1,8,0.0,0


Las siguientes variables contienen la información preprocesada:

In [17]:
captions = pd.read_csv('Data/Features/caption_features.csv')
colors = pd.read_csv('Data/Features/colors_features.csv')
audio_embeddings = pd.read_csv('Data/Features/audio_vectors.csv').to_numpy()[:,1:].reshape((-1,20,1024))

> **Importante:** Note que `audio_embeddings` es un tensor y no un `pd.DataFrame`. Esto se debe a que las dimensiones de este tipo de dato son multidimensionales y los dataframes en `pandas` no permiten este tipo de datos.

In [ ]:
# completar solución